In [2]:
import pickle
test1 = pickle.load(open('./pickled_objects/test1.p','rb'))
probssentences = pickle.load(open('./pickled_objects/probssentences.p', 'rb'))
probssentences_train = pickle.load(open('./pickled_objects/probssentences_train.p', 'rb'))
train1 = pickle.load(open('./pickled_objects/train1.p','rb'))

### Now do the same for the article replication dataset

In [2]:
import pickle
test1 = pickle.load(open('./pickled_objects/articleTest1.p','rb'))
probssentences = pickle.load(open('./pickled_objects/probssentencesArticle.p', 'rb'))
probssentences_train = pickle.load(open('./pickled_objects/probssentences_train_Article.p', 'rb'))
train1 = pickle.load(open('./pickled_objects/articleTrain1.p','rb'))

In [5]:
#last sentence only
import numpy as np
preds = []
for doc in np.unique(probssentences["doc"]):
    inx = np.where(probssentences["doc"] == doc)[0][-1]
    if probssentences[0][inx] > probssentences[1][inx]:
        preds.append(0)
    else:
        preds.append(1)
    

In [3]:
#standard averaging
import numpy as np
preds = []
for doc in np.unique(probssentences["doc"]):
    inx = np.where(probssentences["doc"] == doc)[0]
    if np.mean(probssentences[0][inx]) > np.mean(probssentences[1][inx]):
        preds.append(0)
    else:
        preds.append(1)


In [14]:
#testing
import numpy as np
preds = []
i = 0
for doc in np.unique(probssentences["doc"]):
    if i == 0:
        inx = np.where(probssentences["doc"] == doc)[0]
        print probssentences.loc[probssentences["doc"] == doc]
        if np.mean(probssentences[0][inx]) > np.mean(probssentences[1][inx]):
            preds.append(0)
        else:
            preds.append(1)
        print preds
        print test1['boolean_label'].iloc[doc]
    i = i + 1


            0         1 doc
0    0.603779  0.396221   0
1   0.0390125  0.960988   0
2   0.0562234  0.943777   0
3  0.00171279  0.998287   0
4    0.114009  0.885991   0
5    0.350237  0.649763   0
6    0.580291  0.419709   0
[1]
0


In [7]:
#max
import numpy as np
preds = []
for doc in np.unique(probssentences["doc"]):
    inx = np.where(probssentences["doc"] == doc)[0]
    if np.max(probssentences[0][inx]) > np.max(probssentences[1][inx]):
        preds.append(0)
    else:
        preds.append(1)


In [11]:
#apply sigmoidal function
import numpy as np
preds = []
k=20
for doc in np.unique(probssentences["doc"]):
    inx = np.where(probssentences["doc"] == doc)[0]
    scores0 = probssentences[0][inx].astype(float)-0.5
    scores0 = 1./(1+np.exp(-k*scores0))
    scores1 = probssentences[1][inx].astype(float)-0.5
    scores1 = 1./(1+np.exp(-k*scores1))
    if np.mean(scores0) > 0.5:
        preds.append(0)
    else:
        preds.append(1)

In [56]:
scores0

0    0.006922
1    0.009880
2    0.293727
3    0.006781
4    0.007921
5    0.009454
6    0.008236
7    0.006718
Name: 0, dtype: float64

In [57]:
scores1

0    0.993078
1    0.990120
2    0.706273
3    0.993219
4    0.992079
5    0.990546
6    0.991764
7    0.993282
Name: 1, dtype: float64

In [82]:
features = np.zeros((np.unique(probssentences["doc"]).shape[0],4))
features.shape

(12500, 4)

In [5]:
from sklearn.metrics import roc_auc_score
#print roc_auc_score(test1["sentiment"], preds)
print roc_auc_score(test1["boolean_label"], preds)

0.493068035944


## Decision Tree to see rule for selecting methods

In [28]:
import numpy as np

def getFeatures(probssentences_train, train1):
    features = np.zeros((np.unique(probssentences_train["doc"]).shape[0],11))
    label = []
    for doc in np.unique(probssentences_train["doc"]):
        inx = np.where(probssentences_train["doc"] == doc)[0]
        subset = probssentences_train[0].iloc[inx]
        score = train1['sentiment'].iloc[doc]
        label.append(score)

        inxFirst = inx[0]
        inxLast = inx[-1]

        features[doc,0] = np.mean(subset) #mean of scores
        features[doc,1] = np.min(subset) #min score
        features[doc,2] = np.max(subset) #max score
        features[doc,3] = np.std(subset) #stdev of scores
        features[doc,4] = probssentences_train[0].iloc[inxFirst] #score of first sentence
        features[doc,5] = probssentences_train[0].iloc[inxLast] #score of last sentences
        features[doc,6] = len(subset) #number of sentences
        features[doc,7] = np.mean(                               #sigmoid (with k = 10)
                            1./(1+np.exp(-10*
                               subset.astype(float)-0.5 ))) 
        features[doc,8] = np.mean(                               #sigmoid (with k = 20)
                            1./(1+np.exp(-20*
                               subset.astype(float)-0.5 )))
        features[doc,9] = np.argmin(subset)-inxFirst #position of min score
        features[doc,10] = np.argmax(subset)-inxFirst #position of max score
    
    return (features, label)

### Testing- ignore the below

In [9]:
import numpy as np
featuresTest = np.zeros((np.unique(probssentences_train["doc"]).shape[0],11))
for doc in np.unique(probssentences_train["doc"]):
    if doc == 6:
        inx = np.where(probssentences_train["doc"] == doc)[0]
        subset = probssentences_train[0].iloc[inx]

        print probssentences_train[0].iloc[inx]
        inxFirst = inx[0]
        inxLast = inx[-1]
        
        print np.argmin(subset)-inxFirst
        print inxFirst

        featuresTest[doc,0] = np.mean(subset) #mean of scores
        featuresTest[doc,1] = np.min(subset) #min score
        featuresTest[doc,2] = np.max(subset) #max score
        featuresTest[doc,3] = np.std(subset) #stdev of scores
        featuresTest[doc,4] = probssentences_train[0].iloc[inxFirst] #score of first sentence
        featuresTest[doc,5] = probssentences_train[0].iloc[inxLast] #score of last sentences
        featuresTest[doc,6] = len(subset) #number of sentences
        featuresTest[doc,7] = np.mean(                               #sigmoid (with k = 10)
                            1./(1+np.exp(-10*
                               subset.astype(float)-0.5 ))) 
        featuresTest[doc,8] = np.mean(                               #sigmoid (with k = 20)
                            1./(1+np.exp(-20*
                               subset.astype(float)-0.5 )))
        featuresTest[doc,9] = np.argmin(subset)-inxFirst #position of min score
        featuresTest[doc,10] = np.argmax(subset)-inxFirst #position of max score 

50       0.222634
51      0.0091073
52      0.0472875
53      0.0309574
54    2.94676e-06
55      0.0222528
56      0.0509557
57      0.0187423
58      0.0193402
59       0.184928
Name: 0, dtype: object
4
50


In [10]:
print featuresTest[6]

[  6.06208684e-02   2.94675715e-06   2.22634107e-01   7.35594841e-02
   2.22634107e-01   1.84928447e-01   1.00000000e+01   7.27344256e-01
   7.78256602e-01   4.00000000e+00   0.00000000e+00]


## Build Decision Tree

In [29]:
obj = getFeatures(probssentences_train, train1)
features = obj[0]
label = obj[1]
print len(features)
#df = pd.DataFrame(features, columns=['mean0', 'min0', 'max0', 'stdev0', 'firstsentscore', 'lastsentscore', 
                                     #'numsents', 'sigmoid10', 'sigmoid20', 'posminscore', 'posmaxscore'])
print len(label)

12500
12500


In [35]:
print features[6]
print label[6]
print probssentences_train[probssentences_train["doc"] == 6]
print train1['sentiment'].iloc[6]

[  6.06208684e-02   2.94675715e-06   2.22634107e-01   7.35594841e-02
   2.22634107e-01   1.84928447e-01   1.00000000e+01   7.27344256e-01
   7.78256602e-01   4.00000000e+00   0.00000000e+00]
1
              0         1 doc
50     0.222634  0.777366   6
51    0.0091073  0.990893   6
52    0.0472875  0.952713   6
53    0.0309574  0.969043   6
54  2.94676e-06  0.999997   6
55    0.0222528  0.977747   6
56    0.0509557  0.949044   6
57    0.0187423  0.981258   6
58    0.0193402   0.98066   6
59     0.184928  0.815072   6
1


In [36]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(min_samples_leaf=1000)
#criterion = 'entropy
clf = clf.fit(features, label)

In [39]:
resultTrain = clf.predict(features)

from sklearn.metrics import roc_auc_score
print roc_auc_score(label, resultTrain)

0.973028412975


In [40]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier()
forest = forest.fit( features, label )

In [68]:
clf.feature_importances_

array([  9.99690365e-01,   1.57310033e-04,   1.45630794e-04,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         1.63611859e-06,   9.70464625e-07,   0.00000000e+00,
         0.00000000e+00,   4.08789830e-06])

In [69]:
#export the tree
with open('tree1.dot', 'w') as dotfile:
    tree.export_graphviz(
        clf,
        dotfile,
        feature_names=['mean0', 'min0', 'max0', 'stdev0', 'firstsentscore', 'lastsentscore', 
                                     'numsents', 'sigmoid10', 'sigmoid20', 'posminscore', 'posmaxscore'],
        class_names=['0', '1']
        )

## Attempts to visualize tree

In [19]:
def get_code(tree, feature_names):
        left      = tree.tree_.children_left
        right     = tree.tree_.children_right
        threshold = tree.tree_.threshold
        features  = [feature_names[i] for i in tree.tree_.feature]
        value = tree.tree_.value

        def recurse(left, right, threshold, features, node):
                if (threshold[node] != -2):
                        print "if ( " + features[node] + " <= " + str(threshold[node]) + " ) {"
                        if left[node] != -1:
                                recurse (left, right, threshold, features,left[node])
                        print "} else {"
                        if right[node] != -1:
                                recurse (left, right, threshold, features,right[node])
                        print "}"
                else:
                        print "return " + str(value[node])

        recurse(left, right, threshold, features, 0)

In [23]:
get_code(clf, ['mean0', 'min0', 'max0', 'stdev0', 'firstsentscore', 'lastsentscore', 'numsents', 'sigmoid10', 'sigmoid20', 'posminscore', 'posmaxscore'])

if ( mean0 <= 0.496581494808 ) {
if ( mean0 <= 0.409248709679 ) {
if ( min0 <= 0.00446635391563 ) {
if ( stdev0 <= 0.474251747131 ) {
if ( stdev0 <= 0.356082618237 ) {
if ( numsents <= 2.5 ) {
if ( firstsentscore <= 0.0210778415203 ) {
if ( min0 <= 1.29640611704e-05 ) {
return [[   0.  147.]]
} else {
if ( sigmoid10 <= 0.622507095337 ) {
return [[ 1.  0.]]
} else {
return [[  0.  60.]]
}
}
} else {
if ( firstsentscore <= 0.0279007069767 ) {
return [[ 1.  0.]]
} else {
if ( min0 <= 0.00118211214431 ) {
return [[  0.  21.]]
} else {
if ( sigmoid20 <= 0.737216174603 ) {
return [[ 1.  0.]]
} else {
return [[ 0.  3.]]
}
}
}
}
} else {
if ( stdev0 <= 0.305830955505 ) {
if ( mean0 <= 0.327598065138 ) {
if ( sigmoid10 <= 0.827702045441 ) {
return [[    0.  3414.]]
} else {
if ( sigmoid10 <= 0.827731609344 ) {
return [[ 1.  0.]]
} else {
return [[   0.  446.]]
}
}
} else {
if ( mean0 <= 0.328518509865 ) {
return [[ 1.  0.]]
} else {
return [[  0.  50.]]
}
}
} else {
if ( stdev0 <= 0.30585765838

## Scoring decision tree on test data's sentences

In [45]:
test = getFeatures(probssentences, test1)
featuresT = test[0]
labelsT = test[1]

In [43]:
print featuresT[0]

[[  6.61638838e-02   3.75713920e-04   4.12264854e-01 ...,   7.31899105e-01
    7.00000000e+00   2.00000000e+00]
 [  2.47391960e-01   3.98679589e-07   9.63713348e-01 ...,   7.85682392e-01
    1.00000000e+00   6.00000000e+00]
 [  2.38121294e-01   1.50391394e-02   9.98147368e-01 ...,   8.94100065e-01
    0.00000000e+00   5.00000000e+00]
 ..., 
 [  9.43377061e-01   7.74775147e-01   9.99992132e-01 ...,   9.99999981e-01
    3.00000000e+00   4.00000000e+00]
 [  1.90151211e-01   6.20619346e-07   6.91122890e-01 ...,   8.65802175e-01
    4.00000000e+00   2.00000000e+00]
 [  3.48111241e-01   8.94577242e-03   8.88846219e-01 ...,   8.89102988e-01
    7.00000000e+00   1.00000000e+01]]


In [45]:
print probssentences.iloc[0:8]

             0         1 doc
0   0.00338731  0.996613   0
1    0.0392682  0.960732   0
2     0.412265  0.587735   0
3   0.00132289  0.998677   0
4    0.0169766  0.983023   0
5     0.034818  0.965182   0
6    0.0208975  0.979103   0
7  0.000375714  0.999624   0


In [46]:
result = clf.predict(featuresT)

from sklearn.metrics import roc_auc_score
print roc_auc_score(labelsT, result)

0.867838314629


In [47]:
clf2 = forest.predict(featuresT)
print roc_auc_score(labelsT, clf2)

0.868909594148


### Unsuccessful attempts to visualize tree

In [23]:
from sklearn.externals.six import StringIO  
import pydotplus
dot_data = StringIO() 
tree.export_graphviz(clf, out_file=dot_data) 
graph = pydotplus.graph_from_dot_data(dot_data.getvalue()) 
graph.write_pdf("tree.pdf") 

InvocationException: Program terminated with status: 1. stderr follows: Format: "pdf" not recognized. Use one of: canon cmap cmapx cmapx_np dot eps fig gv imap imap_np ismap pic plain plain-ext pov ps ps2 svg svgz tk vml vmlz xdot xdot1.2 xdot1.4


In [30]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn import tree

iris = load_iris()
test_idx = [0, 50, 100]

# training data
train_target = np.delete(iris.target, test_idx)
train_data = np.delete(iris.data, test_idx, axis=0)

# testing data
test_target = iris.target[test_idx]
test_data = iris.data[test_idx]

clf = tree.DecisionTreeClassifier()
clf.fit(train_data, train_target)

print test_target
print clf.predict(test_data) 

# viz code
from sklearn.externals.six import StringIO
import pydot
dot_data = StringIO()
tree.export_graphviz(clf,
        out_file=dot_data,
        feature_names=iris.feature_names,
        class_names=iris.target_names,
        filled=True, rounded=True,
        impurity=False)

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
print type(graph)
graph.write_pdf("iris.pdf")

[0 1 2]
[0 1 2]
<class 'pydotplus.graphviz.Dot'>


InvocationException: Program terminated with status: 1. stderr follows: Format: "pdf" not recognized. Use one of: canon cmap cmapx cmapx_np dot eps fig gv imap imap_np ismap pic plain plain-ext pov ps ps2 svg svgz tk vml vmlz xdot xdot1.2 xdot1.4


In [10]:
from sklearn.externals.six import StringIO  
import pydot
dot_data = StringIO() 
tree.export_graphviz(clf, out_file=dot_data) 
graph = pydot.graph_from_dot_data(dot_data.getvalue()) 
print type(graph)
#graph.write_pdf("tree.pdf")

<type 'list'>


In [92]:
from IPython.display import Image  
from sklearn.externals.six import StringIO 
import pydot 
dot_data = StringIO()  
tree.export_graphviz(clf, out_file=dot_data,  
                         #feature_names=iris.feature_names,  
                         #class_names=[0,1],  
                         filled=True, rounded=True,  
                         special_characters=True)  
graph = pydot.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())  

KeyboardInterrupt: 

In [9]:
import numpy as np

estimator = clf

# The decision estimator has an attribute called tree_  which stores the entire
# tree structure and allows access to low level attributes. The binary tree
# tree_ is represented as a number of parallel arrays. The i-th element of each
# array holds information about the node `i`. Node 0 is the tree's root. NOTE:
# Some of the arrays only apply to either leaves or split nodes, resp. In this
# case the values of nodes of the other type are arbitrary!
#
# Among those arrays, we have:
#   - left_child, id of the left child of the node
#   - right_child, id of the right child of the node
#   - feature, feature used for splitting the node
#   - threshold, threshold value at the node
#

# Using those arrays, we can parse the tree structure:

n_nodes = estimator.tree_.node_count
children_left = estimator.tree_.children_left
children_right = estimator.tree_.children_right
feature = estimator.tree_.feature
threshold = estimator.tree_.threshold


# The tree structure can be traversed to compute various properties such
# as the depth of each node and whether or not it is a leaf.
node_depth = np.zeros(shape=n_nodes)
is_leaves = np.zeros(shape=n_nodes, dtype=bool)
stack = [(0, -1)]  # seed is the root node id and its parent depth
while len(stack) > 0:
    node_id, parent_depth = stack.pop()
    node_depth[node_id] = parent_depth + 1

    # If we have a test node
    if (children_left[node_id] != children_right[node_id]):
        stack.append((children_left[node_id], parent_depth + 1))
        stack.append((children_right[node_id], parent_depth + 1))
    else:
        is_leaves[node_id] = True

print("The binary tree structure has %s nodes and has "
      "the following tree structure:"
      % n_nodes)
for i in range(n_nodes):
    if is_leaves[i]:
        print("%snode=%s leaf node." % (node_depth[i] * "\t", i))
    else:
        print("%snode=%s test node: go to node %s if X[:, %s] <= %ss else to "
              "node %s."
              % (node_depth[i] * "\t",
                 i,
                 children_left[i],
                 feature[i],
                 threshold[i],
                 children_right[i],
                 ))
print()

# First let's retrieve the decision path of each sample. The decision_path
# method allows to retrieve the node indicator functions. A non zero element of
# indicator matrix at the position (i, j) indicates that the sample i goes
# through the node j.

node_indicator = estimator.decision_path(X_test)

# Similarly, we can also have the leaves ids reached by each sample.

leave_id = estimator.apply(X_test)

# Now, it's possible to get the tests that were used to predict a sample or
# a group of samples. First, let's make it for the sample.

sample_id = 0
node_index = node_indicator.indices[node_indicator.indptr[sample_id]:
                                    node_indicator.indptr[sample_id + 1]]

print('Rules used to predict sample %s: ' % sample_id)
for node_id in node_index:
    if leave_id[sample_id] != node_id:
        continue

    if (X_test[sample_id, feature[node_id]] <= threshold[node_id]):
        threshold_sign = "<="
    else:
        threshold_sign = ">"

    print("decision id node %s : (X[%s, %s] (= %s) %s %s)"
          % (node_id,
             sample_id,
             feature[node_id],
             X_test[i, feature[node_id]],
             threshold_sign,
             threshold[node_id]))

# For a group of samples, we have the following common node.
sample_ids = [0, 1]
common_nodes = (node_indicator.toarray()[sample_ids].sum(axis=0) ==
                len(sample_ids))

common_node_id = np.arange(n_nodes)[common_nodes]

print("\nThe following samples %s share the node %s in the tree"
      % (sample_ids, common_node_id))
print("It is %s %% of all nodes." % (100 * len(common_node_id) / n_nodes,))

The binary tree structure has 12263 nodes and has the following tree structure:
node=0 test node: go to node 1 if X[:, 2] <= 12220.5s else to node 11990.
	node=1 test node: go to node 2 if X[:, 1] <= 12076.5s else to node 11817.
		node=2 test node: go to node 3 if X[:, 0] <= 12069.5s else to node 11812.
			node=3 test node: go to node 4 if X[:, 1] <= 1.5s else to node 5.
				node=4 leaf node.
				node=5 test node: go to node 6 if X[:, 1] <= 3.5s else to node 7.
					node=6 leaf node.
					node=7 test node: go to node 8 if X[:, 0] <= 9669.5s else to node 9431.
						node=8 test node: go to node 9 if X[:, 1] <= 9658.5s else to node 9422.
							node=9 test node: go to node 10 if X[:, 2] <= 7739.5s else to node 7563.
								node=10 test node: go to node 11 if X[:, 1] <= 7085.5s else to node 6954.
									node=11 test node: go to node 12 if X[:, 1] <= 7080.5s else to node 6953.
										node=12 test node: go to node 13 if X[:, 1] <= 3750.5s else to node 3714.
											node=13 test nod

/home/sarahwie/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:53: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/sarahwie/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:49: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


																						node=9579 test node: go to node 9580 if X[:, 1] <= 9817.5s else to node 9581.
																							node=9580 leaf node.
																							node=9581 leaf node.
																					node=9582 test node: go to node 9583 if X[:, 2] <= 9822.5s else to node 9584.
																						node=9583 leaf node.
																						node=9584 test node: go to node 9585 if X[:, 2] <= 9824.5s else to node 9586.
																							node=9585 leaf node.
																							node=9586 test node: go to node 9587 if X[:, 2] <= 9825.5s else to node 9588.
																								node=9587 leaf node.
																								node=9588 leaf node.
										node=9589 leaf node.
									node=9590 test node: go to node 9591 if X[:, 2] <= 9834.5s else to node 9592.
										node=9591 leaf node.
										node=9592 test node: go to node 9593 if X[:, 1] <= 9835.5s else to node 9594.
											node=9593 leaf node.
											node=9594 test node: go to node 9595 if X[:, 1] <= 9837

AttributeError: 'DecisionTreeClassifier' object has no attribute 'decision_path'